In [1]:
#install and import require libraries
!pip install setuptools packaging pandas dash plotly more-itertools
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 46.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 1.3 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.3/227.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 2.8 MB/s eta 0:00:00


In [2]:
# Load the data using pandas
data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')

In [3]:
# Initialize the Dash app
app = dash.Dash(__name__)

In [4]:
# Set the title of the dashboard
app.title = "Automobile Statistics Dashboard"

In [5]:
# Create the dropdown menu options
dropdown_options = [
    {'label': 'Yearly Statistics Report', 'value': 'Yearly Statistics'},
    {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
]

In [6]:
# List of years
year_list = [str(i) for i in range(1980, 2024)]

In [7]:
# Create the layout of the app
app.layout = html.Div([
    #Title
    html.H1("Automobile Statistics Dashboard", 
    style={'textAlign': 'center', 'color': '#503D36', 'font-size': 24}),
    #First Dropdown
    html.Div([
        html.Label("Select Statistics:"),
        dcc.Dropdown(
            id='dropdown-statistics',
            options=dropdown_options,
            value='Select statistics',
            placeholder='Select a report'
        )
    ]),
    #Second Dropdown
    html.Div(dcc.Dropdown(
        id='select-year',
        options=[{'label': i, 'value': i} for i in year_list],
        value='Select Year',
        style={'width': '80%', 'padding': 3, 'textAlign': 'center', 'font-size': 20}
    )),
    #Output result layout
    html.Div([
        html.Div(id='output-container', className='chart-grid', style={'display': 'flex'})
    ])
])

In [8]:
# Callback to update the input container based on the selected statistics
@app.callback(
    Output(component_id='select-year', component_property='disabled'),
    Input(component_id='dropdown-statistics', component_property='value')
)

#Disable the select-year dropdown feature when choose Recession Period Statistics on the first dropdown will return True to the app.callback above
def update_input_container(selected_statistics):
    if selected_statistics == 'Yearly Statistics':
        return False
    else:
        return True

In [9]:
# Callback for plotting
@app.callback(
    Output(component_id='output-container', component_property='children'),
    [Input(component_id='select-year', component_property='value'),
     Input(component_id='dropdown-statistics', component_property='value')]
)
# Plot visualizations for recession period statistics
def update_output_container(selected_year, selected_statistics):
    try:
        if selected_statistics == 'Recession Period Statistics':
            # Filter the data for recession periods
            recession_data = data[data['Recession'] == 1]

            #Plot 1 Automobile sales fluctuate over Recession Period (year wise) using line chart
            yearly_rec = recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
            R_chart1 = dcc.Graph(
                figure=px.line(yearly_rec,
                               x='Year',
                               y='Automobile_Sales',
                               title="Average Automobile Sales fluctuation over Recession Period"))

            #Plot 2 Calculate the average number of vehicles sold by vehicle type and represent as a Bar chart
            average_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
            R_chart2 = dcc.Graph(
                figure=px.bar(average_sales,
                              x='Vehicle_Type',
                              y='Automobile_Sales',
                              title="Average Automobile Sales by Vehicle Type during Recession Period"))

            # Plot 3 : Pie chart for total expenditure share by vehicle type during recessions
            exp_rec = recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
            R_chart3 = dcc.Graph(
                figure=px.pie(exp_rec,
                              values='Advertising_Expenditure',
                              names='Vehicle_Type',
                              title="Total Expenditures by Vehicle Type during Recession Period"))

            # Plot 4 Develop a Bar chart for the effect of unemployment rate on vehicle type and sales
            unemp_rate = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
            R_chart4 = dcc.Graph(
                figure=px.bar(unemp_rate,
                              x='Vehicle_Type',
                              y='Automobile_Sales',
                              title="Effects of Unemployment Rate on Automobile Sales by Vehicle Type during Recession Period"))

            return [
                html.Div(className='chart-item',
                         children=[html.Div(children=R_chart1), html.Div(children=R_chart2)],
                         style={'display': 'flex'}),
                html.Div(className='chart-item',
                         children=[html.Div(children=R_chart3), html.Div(children=R_chart4)],
                         style={'display': 'flex'})
            ]

        elif selected_statistics == 'Yearly Statistics' and selected_year != 'Select Year':
            #Set selected_year to int
            Seletec_Year_Int = int(selected_year)

            # Filter the data for the selected year
            yearly_data = data[data['Year'] == Seletec_Year_Int]

            # Plot 1 :Yearly Automobile sales using line chart for the whole period.
            yas = data.groupby('Year')['Automobile_Sales'].sum().reset_index()
            Y_chart1 = dcc.Graph(
                figure=px.line(yas,
                               x='Year',
                               y='Automobile_Sales',
                               title='Automobile Sales for the Year'))

            # Plot 2 :Total Monthly Automobile sales using line chart.
            mas = yearly_data.groupby('Month')['Automobile_Sales'].sum().reset_index()
            Y_chart2 = dcc.Graph(
                figure=px.line(mas,
                               x='Month',
                               y='Automobile_Sales',
                               title='Total Automobile Sales by Month'))

            # Plot 3 :bar chart for average number of vehicles sold during the given year
            exp_data = yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
            Y_chart3 = dcc.Graph(
                figure=px.bar(exp_data,
                              x='Vehicle_Type',
                              y='Advertising_Expenditure',
                              title='Total Advertising Expenditure by Vehicle Type'))

            # Plot 4 :Total Advertisement Expenditure for each vehicle using pie chart
            avr_vdata = yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
            Y_chart4 = dcc.Graph(
                figure=px.bar(avr_vdata,
                              x='Vehicle_Type',
                              y='Automobile_Sales',
                              title='Average Vehicles Sold by Vehicle Type in the year {}'.format(Seletec_Year_Int)))

            return [
                html.Div(className='chart-item',
                         children=[html.Div(children=Y_chart1), html.Div(children=Y_chart2)],
                         style={'display': 'flex'}),
                html.Div(className='chart-item',
                         children=[html.Div(children=Y_chart3), html.Div(children=Y_chart4)],
                         style={'display': 'flex'})
            ]

        else:
            return None
    except Exception as e:
        return f"Error: {str(e)}"

In [10]:
# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)